In [3]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import csv

In [75]:
#Functions needed for cointegration test and statistical tests
limit=5000
def get_price(tick,res):
    res=str(res)
    hist=requests.get('https://ftx.com/api/markets/'+tick+'-PERP/candles?resolution='+res+'&start_time='+'0'+'&limit='+str(limit)).json()
    hist=pd.DataFrame(hist['result'])
    hist.drop(['startTime'], axis = 1, inplace=True)
    return hist

def get_time(hist):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    aux.set_index('time', inplace=True)
    return aux

def get_initime(hist):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    return aux['time'][0]

def get_time(hist,i):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    return aux['time'][i]
def coint_test(tick1,tick2,res):
    A=get_price(tick1,res)
    B=get_price(tick2,res)
    if not get_initime(B)==get_initime(A):
        print('Different initial time')
        return 
    X = np.stack((np.log10(A['close']), np.log10(B['close'])), axis=0)
    cov=np.cov(X)
    #print(cov)
    covAB=cov[0,1]
    varA=cov[0,0]
    varB=cov[1,1]
    cor=covAB/(np.sqrt(varA*varB))
    lAB=covAB/varB #res=log(A)-lAB*log(B)
    lBA=covAB/varA #res=log(B)-lBA*log(A)
    spreadAB=np.log10(A['close'])-lAB*np.log10(B['close'])
    muAB=np.mean(spreadAB)
    spreadBA=np.log10(B['close'])-lBA*np.log10(A['close'])
    muBA=np.mean(spreadBA)
    return cor,lAB,lBA,muAB,muBA,spreadAB,spreadBA

def print_spread(tick1,tick2,res):
    aux=coint_test(tick1,tick2,res)
    if aux[1]>aux[2]:
        print('We are gonna use A-lAB*B with correlation='+str(aux[0])+' and lambda_AB='+str(aux[1]))
        plt.plot(aux[5]-aux[3])
        return aux[5]
    else :
        print('We are gonna use B-lBA*A with correlation='+str(aux[0])+' and lambda_BA='+str(aux[2]))
        plt.plot(aux[6]-aux[4])
        return aux[6]

In [63]:
#Export pairs in the .csv file from the ADF test
with open('pair_adf.csv', newline='') as f:
    reader = csv.reader(f)
    pair_adf = list(reader)

In [76]:
res=3600
pair_info=[]
for tick1,tick2 in pair_adf:
    #For some reason the first character of tick2 is a space
    tick2=tick2[1:]
    try:
        aux=coint_test(tick1,tick2,res)
        pair_info.append((tick1,tick2,aux[0],aux[1],aux[2]))
        print(tick1,tick2,aux[0])
    except:
        pass

1INCH ALCX 0.6027018460656426
1INCH ALT 0.8051653996837743
1INCH AMPL 0.4740299559225104
1INCH ASD 0.7858966872196413
1INCH AUDIO 0.6439265332574635
1INCH BADGER 0.8000286884800168
1INCH BSV 0.7710721664788182
1INCH BTC 0.7988341502628996
1INCH CHZ 0.828287051855846
1INCH DENT 0.8125571071946298
1INCH DRGN 0.858393021324854
1INCH GRT 0.8641844425050241
1INCH HOT 0.8489780529756733
1INCH KNC 0.7239602711373999
1INCH KSM 0.8016784133861932
1INCH LINA 0.7086830398460611
1INCH LINK 0.8589630401544072
1INCH LTC 0.8879572796978924
1INCH MEDIA 0.49057012542206674
1INCH MTA 0.5754223271289765
1INCH MTL 0.7186993465901225
1INCH ONT 0.7849821017239116
1INCH PROM 0.7424073616185525
1INCH QTUM 0.8313200279485963
1INCH REEF 0.9318757689417224
1INCH ROOK 0.7670413304310345
1INCH RUNE 0.9166722349543407
1INCH SC 0.7521849568225264
1INCH SKL 0.7521691110886549
1INCH THETA 0.6276761959340355
1INCH TRU 0.7346074042312128
1INCH TRX 0.7664970313343001
1INCH VET 0.9031325171128027
1INCH XLM 0.8510435459148

ALT SC 0.9088835880191237
ALT STMX 0.8981297550936058
ALT TRU 0.8954753928303552
ALT ZRX 0.9186078137655084
AMPL ASD 0.4846467737175572
AMPL BAL 0.2970482899286211
AMPL BAND 0.4056243400053351
AMPL BCH 0.25379302657580266
AMPL BNT 0.357649034815192
AMPL BRZ -0.3278838314768107
AMPL BSV 0.3473988226412681
AMPL BTC 0.4439607812116947
AMPL BTT 0.2225769307167583
AMPL CAKE 0.22858360479487166
AMPL CHZ 0.30829684856001466
AMPL CONV 0.26449373363701334
AMPL CUSDT 0.14113904175678263
AMPL DASH 0.2732602208686367
AMPL DAWN 0.37440491281684507
AMPL DEFI 0.38708638580887
AMPL DOGE 0.19318789892328456
AMPL DRGN 0.35749382314354894
AMPL EOS 0.22024415596569144
AMPL ETC 0.2300426423976506
AMPL EXCH 0.30288713516538124
AMPL FLM 0.22281025765479037
AMPL GRT 0.42402452657252576
AMPL HBAR 0.33380585030791937
AMPL HOT 0.3035965044820586
AMPL HT -0.15299003032420544
AMPL ICP 0.1348734409812909
AMPL IOTA 0.18398111897552827
AMPL KAVA 0.2797537640894962
AMPL KNC 0.2880554268357288
AMPL KSM 0.27545796625224

BAND DAWN 0.7866262521694799
BAND DOGE 0.7005648731611063
BAND EOS 0.862003396664723
BAND ETC 0.804187426525727
BAND FIL 0.8077896743121646
BAND FLM 0.91268705659369
BAND FTT 0.8246765601223129
BAND GRT 0.8526343044997027
BAND ICP 0.7746992140143334
BAND KAVA 0.8719647351019713
BAND KIN 0.7047146012305755
BAND KSM 0.7465352124094602
BAND LINA 0.767146415236315
BAND LINK 0.9100980721623132
BAND MAPS 0.9054145081424334
BAND MEDIA 0.5280559877876384
BAND MER 0.40757716632487256
BAND MTA 0.46582522958578815
BAND MTL 0.8685215283467816
BAND NEO 0.8804038871656206
BAND ONT 0.9351048937169776
BAND ORBS 0.8146127969725764
BAND OXY 0.8236958992584158
BAND PERP 0.8433766212686024
BAND PRIV 0.8640933463395764
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initial time
Different initia

CONV DAWN 0.49234095394342253
CONV DODO 0.762127205512051
CONV DOGE 0.8692430969238595
CONV EOS 0.8652368717399008
CONV ETC 0.8623389550507528
CONV FIL 0.8568022113836038
CONV FLOW 0.5217959605094294
CONV ICP 0.8517015985080248
CONV KAVA 0.6016905871417556
CONV KIN 0.27117505626468685
CONV MER 0.7716646403687556
CONV NEO 0.8653112111507959
CONV OXY 0.7743072211611371
CONV PERP 0.4473443719568045
CONV PROM 0.6353633662898504
CONV PUNDIX 0.6081745789811189
CONV RAMP 0.4824419674530385
CONV SKL 0.7236771248968211
CONV SNX 0.7318553985126851
CONV SRN 0.8331922445006359
CONV STMX 0.4271511552421303
CONV SUSHI 0.6621157245643146
CONV SXP 0.6769498567691432
CONV TOMO 0.3150701465294506
CONV TRU 0.41561204429386733
CONV TRYB 0.6595939221155409
CONV UNI 0.7707173673160398
CONV WAVES 0.5570217608199671
CONV XEM 0.6535486510530935
CONV YFII 0.27951333178057486
CRO HUM 0.9393904837999678
CRO LRC 0.9694506798112121
CRO SAND 0.9688128961589099
CRV NEAR 0.894552366882456
CUSDT KNC -0.0951080589072934

EOS SKL 0.8479972850695484
EOS SNX 0.8397472587601245
EOS SRN 0.8795461129932148
EOS SUSHI 0.8320143117750848
EOS SXP 0.8445433009881239
EOS THETA 0.8027928536098741
EOS TOMO 0.47983387843645214
EOS TRU 0.6002626051816228
EOS TRYB 0.6251620397152131
EOS UNI 0.8895611907245019
EOS WAVES 0.687087758230208
EOS XEM 0.8404444349667972
EOS XMR 0.8853831491124187
EOS YFII 0.526828931022302
EOS ZIL 0.9164363613187463
ETC FIL 0.9070152451475103
ETC ICP 0.9496560212862092
ETC KAVA 0.802950739601713
ETC KIN 0.44460033556233863
ETC KNC 0.8450168521884763
ETC MEDIA 0.525783612748847
ETC MER 0.7042250515420213
ETC NEO 0.94585695692604
ETC PERP 0.6277697723238643
ETC PRIV 0.9018824079746556
ETC PROM 0.7117180566919195
ETC PUNDIX 0.8092142673750992
ETC SKL 0.7773696766151782
ETC SNX 0.868017802159201
ETC SRN 0.9181622436884925
ETC SUSHI 0.8009247522211622
ETC SXP 0.8493434611594539
ETC THETA 0.7799897194032092
ETC TOMO 0.5345650036218527
ETC TRYB 0.697808545122713
ETC UNI 0.8857724988169019
ETC XEM 0.

KAVA MER 0.3875054059934345
KAVA MKR 0.650665881986888
KAVA MTL 0.7568263408585033
KAVA NEO 0.8334801652833388
KAVA ONT 0.8147652616749586
KAVA PERP 0.8722720052162904
KAVA PRIV 0.8002616758086677
KAVA PROM 0.835517081362897
KAVA PUNDIX 0.906115379950685
KAVA RAMP 0.7916612496504999
KAVA RSR 0.7684444899221657
KAVA SKL 0.8165995602531652
KAVA SNX 0.8129797477675589
KAVA SRN 0.7021325159636649
KAVA STMX 0.7848726986948794
KAVA SUSHI 0.8600696725104965
KAVA SXP 0.898975572095686
KAVA THETA 0.49481593089532755
KAVA TRU 0.6911855189112042
KAVA UNI 0.8786069709582701
KAVA XEM 0.8448982694551365
KAVA XMR 0.8472509993969793
KAVA YFII 0.8428675819507202
KAVA ZIL 0.7943420736909503
KIN KNC 0.596492403276277
KIN KSM 0.6754583210407549
KIN LINA 0.8381978258710984
KIN LINK 0.7434547512449229
KIN MEDIA 0.2819539697258974
KIN MID 0.8249893214060205
KIN MKR 0.5776084980016805
KIN MTA 0.5040523321754032
KIN MTL 0.8004756251972632
KIN NEO 0.5397242299105733
KIN ONT 0.7508076706792929
KIN ORBS 0.7834498

MER OXY 0.5354527027619939
MER PERP 0.11170928285711858
MER PROM 0.299081253922142
MER RAMP 0.19988393847687733
MER RAY -0.12037306962926354
MER SKL 0.43040525210991815
MER SNX 0.7080076334569045
MER SRM 0.3288827506473469
MER SRN 0.7330341075617026
MER SUSHI 0.506904416005779
MER SXP 0.44880887537204617
MER TOMO -0.028853945770907375
MER TRYB 0.8866427489251006
MER UNI 0.5067909931120667
MER UNISWAP -0.0922059076636148
MER WAVES 0.28953025899985946
MER XEM 0.33506297926947876
MER YFII 0.02095119339845942
MID MTA 0.595519113860975
MID MTL 0.911690792218691
MID ONT 0.9135382843439266
MID PROM 0.8739844033281725
MID ROOK 0.7525246850518779
MID RSR 0.8944773451737632
MID SC 0.922856658608872
MID STMX 0.9131181477531927
MID TRU 0.9083222452753199
MID TRX 0.8961410412566475
MID XLM 0.9338193455724876
MID XRP 0.9574412141722882
MID YFII 0.7231817714420177
MID ZRX 0.9559626052385717
MKR MTA 0.6770452239842164
MKR ONT 0.7355558753191618
MKR PRIV 0.8031948148055
MKR PROM 0.5396814790168598
MKR 

RAMP SC 0.8470808479739735
RAMP SKL 0.8959112753337968
RAMP SRM 0.9371305763700528
RAMP SRN 0.49843282400448674
RAMP STEP 0.8417748226055953
RAMP STMX 0.9331777613563241
RAMP SUSHI 0.7135682030330831
RAMP TRU 0.8676754372432846
RAMP TRX 0.8682268898912628
RAMP UNISWAP 0.8694596889027979
RAMP WAVES 0.9012203895104797
RAMP XEM 0.7956594104175547
RAMP XLM 0.8091947660859323
RAMP XRP 0.902702657656904
RAMP XTZ 0.7782977857011475
RAMP YFII 0.8427108450159569
RAMP ZIL 0.7714963922233543
RAMP ZRX 0.8724569838034377
RAY STEP 0.8851197176928345
RAY TRU 0.8222242768960403
REEF ROOK 0.8091482342370299
REEF RUNE 0.9052864246181742
REEF SC 0.8354167512044395
REEF TRU 0.8379664683961445
REEF VET 0.9331449538201773
REN STEP 0.8815073004577084
REN UNISWAP 0.9307372879855764
REN XTZ 0.9541849871081842
ROOK RSR 0.652165855159721
ROOK RUNE 0.7345587608425167
ROOK SC 0.616686547811406
ROOK SKL 0.6426651091699298
ROOK SRN 0.4524634928689353
ROOK STMX 0.6216850338840122
ROOK SUSHI 0.6324533955805758
ROOK TH

In [77]:
pair_info_df=pd.DataFrame(pair_info)
pair_info_df.columns = ['Ticker1', 'Ticker2','ro','lAB','lBA']
pair_info_df=pair_info_df.sort_values(by=['ro'],ascending=False)
pair_info_df=pair_info_df.reset_index()

In [79]:
pair_info_df.to_csv('pairs_ro.csv',index= False)